<a href="https://colab.research.google.com/github/blueberrywaffle1/solved/blob/main/SKAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1. Import modules

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#sys.path.append('../utils')
#from evaluating import evaluation_change_point # evaluting 코드 > evaluating_change_point
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Activation, Dropout, TimeDistributed
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.signal import medfilt
from sklearn import decomposition
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from itertools import product

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 2. Data loading

1) anomaly-free : Data Obtained from the experiments with normal mode </br>
2) valve1 : 펌프 흐름의 배출 밸브를 닫은 실험 값 <br>
3) valve2 : 펌프로 유입되는 입구 밸브를 닫은 실험 값 <br>
4) other

<a href = "notebooks/EDA.ipynb" > SKAB EDA</a>

In [ ]:
all_files = []
for root, dirs, files in os.walk("/content/drive/MyDrive/SKAB data"):
  for file in files:
    if file.endswith(".csv"):
      all_files.append(os.path.join(root,file))

In [ ]:
# 이상치 데이터
list_of_df = [pd.read_csv(file,
                          sep=';',
                          index_col = 'datetime',
                          parse_dates = True) for file in all_files if 'anomaly-free' not in file]

# 정상 데이터
anomaly_free_df = pd.read_csv([file for file in all_files if 'anomaly-free' in file][0],
                              sep = ';',
                              index_col = 'datetime',
                              parse_dates = True)

In [ ]:
anomaly_free_df.head()

,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure,Temperature,Thermocouple,Voltage,Volume Flow RateRMS
datetime,,,,,,,,
2020-02-08 13:30:47,0.202394,0.275154,2.16975,0.382638,90.6454,26.8508,238.852,122.664
2020-02-08 13:30:48,0.203153,0.277857,2.07999,-0.273216,90.7978,26.8639,227.943,122.338
2020-02-08 13:30:50,0.202054,0.275790,2.52577,0.382638,90.7730,26.8603,223.486,121.338
2020-02-08 13:30:51,0.203595,0.278101,2.49742,0.054711,90.8424,26.8616,244.904,121.664
2020-02-08 13:30:52,0.201889,0.276363,2.29194,0.710565,90.6664,26.8603,239.196,122.000


- data description and visualization

In [ ]:
print(f'Shape of the random dataset: {list_of_df[0].shape}\n')
n_cp = sum([len(df[df.changepoint==1.]) for df in list_of_df])
n_oulier = sum([len(df[df.anomaly==1.]) for df in list_of_df])

Shape of the random dataset: (1140, 10)



In [ ]:
list_of_df[1]

,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure,Temperature,Thermocouple,Voltage,Volume Flow RateRMS,anomaly,changepoint
datetime,,,,,,,,,,
2020-03-09 12:34:37,0.028030,0.040400,1.182850,0.054711,69.9219,24.9122,229.918,32.9875,0.0,0.0
2020-03-09 12:34:38,0.026952,0.038880,1.452680,-0.273216,69.9147,24.9038,219.566,32.0129,0.0,0.0
2020-03-09 12:34:39,0.027958,0.040660,1.023000,0.054711,69.8861,24.9055,237.717,32.9875,0.0,0.0
2020-03-09 12:34:40,0.027277,0.040736,1.227510,-0.273216,69.8397,24.9110,227.621,32.0000,0.0,0.0
2020-03-09 12:34:41,0.027210,0.039489,1.215260,-0.273216,69.9136,24.9110,221.634,32.0129,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2020-03-09 12:54:33,0.027358,0.041076,0.646547,0.054711,70.1018,24.8565,241.861,32.0000,0.0,0.0
2020-03-09 12:54:34,0.027187,0.039463,0.672100,0.382638,70.0783,24.8596,246.590,32.0000,0.0,0.0
2020-03-09 12:54:35,0.027795,0.040085,0.596467,0.054711,70.0489,24.8600,232.107,32.0000,0.0,0.0
